In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
valid_filenames

['000000.ogg',
 '000001.ogg',
 '000002.ogg',
 '000004.ogg',
 '000005.ogg',
 '000007.ogg',
 '000009.ogg',
 '000010.ogg',
 '000012.ogg',
 '000013.ogg',
 '000014.ogg',
 '000015.ogg',
 '000016.ogg',
 '000018.ogg',
 '000019.ogg',
 '000020.ogg',
 '000021.ogg',
 '000022.ogg',
 '000025.ogg',
 '000026.ogg',
 '000027.ogg',
 '000028.ogg',
 '000029.ogg',
 '000030.ogg',
 '000033.ogg',
 '000034.ogg',
 '000035.ogg',
 '000037.ogg',
 '000039.ogg',
 '000040.ogg',
 '000041.ogg',
 '000042.ogg',
 '000043.ogg',
 '000045.ogg',
 '000046.ogg',
 '000047.ogg',
 '000049.ogg',
 '000051.ogg',
 '000052.ogg',
 '000053.ogg',
 '000054.ogg',
 '000055.ogg',
 '000056.ogg',
 '000058.ogg',
 '000059.ogg',
 '000061.ogg',
 '000062.ogg',
 '000063.ogg',
 '000066.ogg',
 '000068.ogg',
 '000069.ogg',
 '000070.ogg',
 '000071.ogg',
 '000072.ogg',
 '000073.ogg',
 '000074.ogg',
 '000075.ogg',
 '000076.ogg',
 '000077.ogg',
 '000078.ogg',
 '000080.ogg',
 '000081.ogg',
 '000082.ogg',
 '000083.ogg',
 '000084.ogg',
 '000085.ogg',
 '000087.o

In [10]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [11]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [12]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [13]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [14]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [15]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [16]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

### Create Folds

In [17]:
df_train['fold'] = 0

In [18]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [19]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [20]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [21]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((128,960))])


In [22]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [23]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
        else:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
            
        audio = np.mean(audio, axis=0, keepdims=True)   ### convert to mono
        return audio

    def resample(self,audio,sr):
        
        audio = torchaudio.transforms.Resample(orig_freq= sr, new_freq=  16000, 
                                    resampling_method = 'sinc_interpolation', 
                                    lowpass_filter_width = 6, 
                                    rolloff = 0.99)(audio)
        return audio

    def makeCQT(self,audio,sr):
        audio = CQT1992v2(hop_length=self.HOP_LEN,n_bins=self.n_bins,bins_per_octave=self.bins_per_octave,
                          sr=sr,verbose=False)(audio)
        
        max_ = audio.max()
        if max_ < 0.001:
            audio[...] = 0
        else:
            audio /= max_
        
        return audio
        
        
    def create_spectrogram(self,audio,sr):
                
        specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sr,
                                                        n_fft=self.N_FFT, 
                                                        win_length=self.N_FFT, 
                                                        hop_length=self.HOP_LEN,
                                                        center=True,
                                                        pad_mode="reflect",
                                                        power=2.0,
                                                        norm='slaney',
                                                        onesided=True,
                                                        n_mels=self.n_mels,
                                                        mel_scale="htk"
                                                       )(audio)

        specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
        specgram = specgram - specgram.min()
        
        max_ = specgram.max()
        if max_ < 0.001:
            specgram[...] = 0
        else:
            specgram /= max_
        return specgram

    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        sr = 16000
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
#         samples = self.load_file(row.filename)
    
        samples = np.stack(samples, axis=0)
        
        
        samples = torch.from_numpy(samples).float().view(1,-1)
        
        if self.audiotfms is not None:
             samples = self.audiotfms(samples.unsqueeze(0)).squeeze(0)

        samples = self.create_spectrogram(samples,sr)
#         samples = self.makeCQT(samples,sr)

        if self.transform is not None:
            samples = self.transform(samples)

            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [24]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [25]:
a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=448,N_FFT=512)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

torch.Size([1, 128, 1072])

In [26]:
def gem_1d_res(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), 2,2).pow(1./p)

def gem_1d(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), (x.size(-1),)).pow(1./p )

#### FastAI model

In [27]:
df_train['genre_id'].nunique()

19

In [28]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [29]:
import timm
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x

In [30]:
# timm.create_model('resnest101e',in_chans=1)

In [31]:
# timm.create_model('resnest50d',in_chans=1)

In [32]:
# timm.create_model('resnext50_32x4d',in_chans=1).conv1.stride

In [33]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [34]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [35]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'
        kernel_type_prev = 'resnest50d_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 32
        n_epochs = 5

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                            )

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnest50d', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv1[0].stride = (1,1)
#         model.conv_stem.stride = (1,1)
#         model.stem[0].stride = (2,2)
        model.fc = nn.Linear(2048,19)
    ### use prev models
#         loc = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')
#         model.load_state_dict(loc,strict=False)
    
        f1_score = F1Score(average="micro")
        
        
        
        ### MixUp() in callbacks
#        ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1,patience=2,min_lr=1e-7)
# GradientAccumulation (n_acc = 2)
        
        learn = Learner(dls, model,opt_func=ranger,loss_func=crit,metrics=[f1_score],cbs=[MixUp(alpha=0.5),
                                                                                          SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
#         learn.load()
        weights = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')['model']
        learn.model.load_state_dict(weights)
#         learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}').cuda()
#         learn.fit(n_epochs, 1e-2)       
#         learn.fit_one_cycle(n_epochs, 1e-2, wd=1e-03)

        learn.fit_flat_cos(n_epochs, 1e-2, wd=1e-03)
#         learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           n_mels=128,HOP_LEN=512,N_FFT=512,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [36]:
df_test_valid.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


In [37]:
df_test_valid

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg
...,...,...,...
5071,6427,006427.ogg,test/006427.ogg
5072,16903,016903.ogg,test/016903.ogg
5073,1731,001731.ogg,test/001731.ogg
5074,12871,012871.ogg,test/012871.ogg


In [38]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.776322,1.663625,0.477147,31:46
1,1.746364,1.547365,0.487946,23:09
2,1.724743,1.551844,0.503516,22:48
3,1.706319,1.476419,0.512305,22:52
4,1.625726,1.317464,0.556504,23:10


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.751164,1.507327,0.489955,25:10
1,1.756911,1.427625,0.523606,23:16
2,1.731681,1.485039,0.514566,23:18
3,1.687945,1.839932,0.418132,23:40
4,1.608618,1.294104,0.569312,23:01


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.773777,1.492492,0.503767,24:19
1,1.757022,1.455155,0.505023,25:24
2,1.726115,1.547850,0.493471,23:37
3,1.724650,1.507469,0.498744,23:24
4,1.632589,1.296934,0.560773,23:38


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.778449,1.631739,0.461944,25:53
1,1.749858,1.596681,0.481035,23:46
2,1.741383,1.702091,0.434815,22:57
3,1.709095,1.706477,0.440090,22:23
4,1.625392,1.331752,0.550364,30:07


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.772002,1.616497,0.473750,28:37
1,1.729327,1.856883,0.391861,24:16
2,1.725589,1.806998,0.405928,22:44
3,1.705800,1.485098,0.507661,22:21
4,1.608792,1.338569,0.552876,22:21


In [ ]:
submission

### OOF

In [39]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [40]:
OOFOut['ID']

0        10150
1        11198
2        13166
3        12275
4        24102
         ...  
19903     2609
19904    13023
19905     2022
19906     9337
19907     6579
Name: ID, Length: 19908, dtype: int64

In [41]:
OOFOut.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
19903,0.196652,0.050795,0.045259,0.101106,0.213958,0.013778,0.019891,0.026922,0.133037,0.083032,0.020696,0.000477,0.000454,0.003618,0.000875,0.085700,0.000134,0.002150,1.464807e-03,2609
19904,0.053320,0.063251,0.028427,0.159180,0.018881,0.191049,0.002509,0.054558,0.054699,0.292152,0.012905,0.003297,0.001188,0.039282,0.013782,0.001404,0.001030,0.008939,1.464169e-04,13023
19905,0.012246,0.671207,0.242715,0.013822,0.001147,0.015877,0.001606,0.018430,0.015848,0.001654,0.001151,0.000675,0.000004,0.002455,0.000890,0.000057,0.000013,0.000202,8.404415e-09,2022
19906,0.321933,0.013572,0.003057,0.032386,0.439676,0.003170,0.101965,0.012842,0.016481,0.003844,0.047495,0.000560,0.000065,0.001106,0.000081,0.000302,0.000148,0.000953,3.646082e-04,9337
19907,0.286121,0.013514,0.003622,0.104764,0.103011,0.013892,0.011464,0.267733,0.013831,0.007499,0.160739,0.006639,0.000108,0.004985,0.000087,0.001001,0.000107,0.000536,3.466301e-04,6579


In [42]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [43]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [44]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.5578661844484629

In [45]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,2113,172,64,168,307,26,375,137,159,119,445,1,0,25,2,31,0,3,6
1,100,1533,807,141,17,133,13,101,243,44,10,4,0,29,46,11,1,13,0
2,26,772,1501,20,7,11,3,19,58,14,1,0,0,5,10,6,1,4,0
3,186,191,85,1077,28,72,41,200,91,40,128,33,11,45,2,0,38,5,3
4,182,22,16,35,1320,8,10,3,35,37,11,0,0,0,0,9,1,6,1
5,20,133,14,55,5,785,4,46,102,43,3,5,0,8,14,4,6,6,0
6,211,17,7,54,13,2,703,10,14,3,27,0,0,5,1,5,0,1,0
7,73,74,6,79,10,45,5,398,28,7,64,37,0,28,2,5,0,2,1
8,47,79,36,25,17,53,1,10,157,10,3,1,1,6,8,3,1,4,0


In [55]:
df = OOFOut.groupby('genre_id').size()
df = 1. / df
df = df / df.mean()

class_weights = df.values
logits_weights = np.log((class_weights).reshape((1, -1)))
probit_weights = scipy.special.softmax(logits_weights,1)

In [56]:
probit_weights

array([[0.00235017, 0.00233194, 0.00279526, 0.00400964, 0.00411011,
        0.00594511, 0.00611123, 0.00691318, 0.00763742, 0.00886654,
        0.00906704, 0.01458053, 0.01768961, 0.02358614, 0.05082648,
        0.07678042, 0.07678042, 0.12443724, 0.55518153]])

In [ ]:
f1_score(OOFOut['genre_id'],OOFOut.iloc[:,:19].idxmax(1),average="micro")

In [ ]:
for a in [13,14,15,16]:
    print(a, OOFOut[OOFOut['genre_id'].isin([a])][a].mean(),OOFOut[~OOFOut['genre_id'].isin([a])][a].mean())

### Mode based sub

In [46]:
kernel_type

'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'

In [47]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [48]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub

In [49]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                    'resnest50d_MonoSpec_Hop512_Mels128',
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [50]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels.csv',index=False)

### Mode based sub: specific folds only

In [53]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                    'resnest50d_MonoSpec_Hop512_Mels128',
                    ]:
        if kernels == 'resnest50d_MonoSpec_Hop512_Mels128':
            if fld in [0,1,2]:
                fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
                out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
        else:
            fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
            out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')

submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [54]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels_bestfoldsonly.csv',index=False)

### OOF analysis

In [60]:
OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/oof_fold_{fld}_probs.npy',allow_pickle=True)

OOF = ([item for sublist in OOF for item in sublist])
OOF = pd.DataFrame(OOF)

In [64]:
OOFOut.shape

(19908, 22)

In [65]:
OOFOut

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,ID,song_id,genre_id
0,0.186431,0.094427,0.027347,0.231648,0.113026,0.100664,0.010008,0.053855,0.093081,0.040313,...,0.000646,0.004884,0.000930,0.000514,6.026998e-03,0.001351,2.716361e-04,10150,10150,7
1,0.040791,0.026577,0.015757,0.008647,0.000918,0.010413,0.009106,0.001414,0.026599,0.742320,...,0.000703,0.033031,0.068590,0.008385,2.056078e-05,0.003460,2.172195e-04,11198,11198,9
2,0.592745,0.137687,0.042475,0.005826,0.035585,0.001507,0.024752,0.004201,0.121191,0.008384,...,0.000002,0.002019,0.000638,0.001946,7.262796e-08,0.000076,3.112055e-06,13166,13166,0
3,0.051737,0.003258,0.002781,0.003078,0.909330,0.001108,0.003489,0.000436,0.010207,0.006795,...,0.000054,0.000021,0.000018,0.003816,4.258877e-05,0.000020,1.299730e-03,12275,12275,4
4,0.048153,0.055578,0.028709,0.660223,0.006836,0.027321,0.008651,0.067138,0.012886,0.005722,...,0.006742,0.005041,0.000397,0.000006,2.814146e-02,0.000703,5.968715e-06,24102,24102,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19903,0.196652,0.050795,0.045259,0.101106,0.213958,0.013778,0.019891,0.026922,0.133037,0.083032,...,0.000454,0.003618,0.000875,0.085700,1.338165e-04,0.002150,1.464807e-03,2609,2609,8
19904,0.053320,0.063251,0.028427,0.159180,0.018881,0.191049,0.002509,0.054558,0.054699,0.292152,...,0.001188,0.039282,0.013782,0.001404,1.029839e-03,0.008939,1.464169e-04,13023,13023,9
19905,0.012246,0.671207,0.242715,0.013822,0.001147,0.015877,0.001606,0.018430,0.015848,0.001654,...,0.000004,0.002455,0.000890,0.000057,1.336984e-05,0.000202,8.404415e-09,2022,2022,1
19906,0.321933,0.013572,0.003057,0.032386,0.439676,0.003170,0.101965,0.012842,0.016481,0.003844,...,0.000065,0.001106,0.000081,0.000302,1.480180e-04,0.000953,3.646082e-04,9337,9337,0


In [141]:
kernel_type_upd = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128' 

In [142]:
fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/fold_{fld}.csv')
fname

,song_id,filename,filepath,target
0,7072,007072.ogg,test/007072.ogg,1
1,10207,010207.ogg,test/010207.ogg,12
2,20008,020008.ogg,test/020008.ogg,3
3,10924,010924.ogg,test/010924.ogg,0
4,21896,021896.ogg,test/021896.ogg,4
...,...,...,...,...
5071,6427,006427.ogg,test/006427.ogg,3
5072,16903,016903.ogg,test/016903.ogg,0
5073,1731,001731.ogg,test/001731.ogg,3
5074,12871,012871.ogg,test/012871.ogg,0


In [143]:
preds_out = pd.DataFrame(np.zeros((5076,19)))
for fld in range(5):
    probs = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/test_fold_{fld}_probs.npy',allow_pickle=True)
    p = pd.DataFrame([item.numpy().ravel() for sublist in probs for item in sublist])
    preds_out += p.values/5

In [144]:
preds_out.shape

(5076, 19)

In [145]:
preds_out['song_id'] = fname['song_id']

In [146]:
preds_out.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,song_id
0,0.100673,0.350939,0.098798,0.148765,0.002588,0.030072,0.046450,0.127451,0.018603,0.001938,0.063865,0.002474,0.000030,0.006520,0.000504,0.000054,0.000013,0.000263,7.771861e-07,7072
1,0.002705,0.003200,0.001199,0.002379,0.000531,0.001060,0.000143,0.000411,0.001572,0.002936,0.000317,0.002575,0.978791,0.001153,0.000047,0.000009,0.000244,0.000520,2.068405e-04,10207
2,0.107433,0.045127,0.018690,0.671405,0.003967,0.003552,0.095415,0.016241,0.006093,0.001186,0.019225,0.002050,0.001713,0.004670,0.000356,0.000013,0.002646,0.000216,1.911204e-06,20008
3,0.408020,0.025308,0.015316,0.013020,0.012741,0.005195,0.459114,0.002896,0.011753,0.022588,0.014176,0.001160,0.000358,0.002901,0.000298,0.001478,0.000022,0.002211,1.444544e-03,10924
4,0.132524,0.109830,0.039943,0.116005,0.340778,0.037484,0.067317,0.009901,0.089593,0.028464,0.003379,0.000535,0.000049,0.006866,0.006166,0.004419,0.002807,0.003881,5.982801e-05,21896


In [148]:
preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,song_id
1,0.002705,0.003200,0.001199,0.002379,0.000531,0.001060,0.000143,0.000411,0.001572,0.002936,0.000317,0.002575,0.978791,0.001153,0.000047,9.310382e-06,0.000244,0.000520,2.068405e-04,10207
21,0.049635,0.006066,0.003671,0.010573,0.001125,0.000320,0.923453,0.000532,0.000986,0.000831,0.002337,0.000064,0.000057,0.000218,0.000064,3.486263e-05,0.000022,0.000011,3.958146e-07,20964
33,0.017538,0.003925,0.002236,0.005466,0.000767,0.001673,0.958764,0.001054,0.002470,0.001708,0.001101,0.000390,0.000011,0.001608,0.000401,6.381835e-04,0.000062,0.000168,2.102749e-05,17118
51,0.073265,0.010238,0.004441,0.007683,0.858817,0.003178,0.004375,0.001323,0.022711,0.007468,0.003839,0.000151,0.000065,0.000239,0.000212,7.604379e-04,0.000181,0.000790,2.638000e-04,17282
55,0.046705,0.005931,0.002876,0.014332,0.001789,0.000386,0.920069,0.001449,0.001717,0.000437,0.003998,0.000073,0.000005,0.000155,0.000009,5.553316e-05,0.000006,0.000009,7.213111e-07,12711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5037,0.018857,0.007552,0.010709,0.007973,0.920818,0.002974,0.001421,0.001385,0.008382,0.010237,0.001683,0.000165,0.000971,0.000149,0.000487,3.159867e-03,0.000874,0.000778,1.425228e-03,15164
5046,0.003659,0.006902,0.004112,0.004209,0.001242,0.000086,0.978274,0.000284,0.000705,0.000290,0.000114,0.000008,0.000014,0.000023,0.000028,1.283970e-05,0.000032,0.000004,3.402550e-08,701
5049,0.001462,0.109202,0.864839,0.004584,0.010433,0.000731,0.002090,0.001694,0.001679,0.001239,0.000308,0.000492,0.000357,0.000122,0.000272,4.048433e-05,0.000220,0.000238,3.643305e-07,5142
5066,0.002593,0.004707,0.001719,0.006540,0.000376,0.001107,0.000117,0.001060,0.001758,0.001168,0.000784,0.000754,0.975777,0.001163,0.000042,2.057024e-06,0.000127,0.000153,5.157387e-05,14882


In [151]:
PL = preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0].reset_index(drop=True)

In [152]:
PL.to_csv(r'C:\Users\Kaggle\Pog_Music_Classification\outputs\resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128\PL.csv')

In [153]:
PL.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,song_id
0,0.002705,0.003200,0.001199,0.002379,0.000531,0.001060,0.000143,0.000411,0.001572,0.002936,0.000317,0.002575,0.978791,0.001153,0.000047,0.000009,0.000244,0.000520,2.068405e-04,10207
1,0.049635,0.006066,0.003671,0.010573,0.001125,0.000320,0.923453,0.000532,0.000986,0.000831,0.002337,0.000064,0.000057,0.000218,0.000064,0.000035,0.000022,0.000011,3.958146e-07,20964
2,0.017538,0.003925,0.002236,0.005466,0.000767,0.001673,0.958764,0.001054,0.002470,0.001708,0.001101,0.000390,0.000011,0.001608,0.000401,0.000638,0.000062,0.000168,2.102749e-05,17118
3,0.073265,0.010238,0.004441,0.007683,0.858817,0.003178,0.004375,0.001323,0.022711,0.007468,0.003839,0.000151,0.000065,0.000239,0.000212,0.000760,0.000181,0.000790,2.638000e-04,17282
4,0.046705,0.005931,0.002876,0.014332,0.001789,0.000386,0.920069,0.001449,0.001717,0.000437,0.003998,0.000073,0.000005,0.000155,0.000009,0.000056,0.000006,0.000009,7.213111e-07,12711


In [158]:
np.argmax(PL.iloc[:,:-1].values,1)

array([12,  6,  6,  4,  6,  4, 12,  4, 12, 12,  4,  4,  4,  4,  4, 12,  6,
       12,  4,  4,  4, 12,  5,  6, 11,  4, 12,  4,  6, 12,  6,  4,  4, 12,
       12, 11, 12,  4,  6,  4,  6, 12,  4,  4,  4, 12, 12,  6,  6,  9, 12,
        6, 12,  4,  4,  4,  5, 11,  5,  6, 12,  3,  5,  6,  5,  5,  6,  9,
        6,  6,  5,  4, 12, 12, 11,  5,  4,  4, 12,  5,  3,  6,  6,  4,  4,
        6,  4,  6, 12,  9, 12,  4,  6,  3,  4,  4,  4,  6,  6, 12,  4, 12,
        6,  4,  4,  3,  4,  4,  4,  4, 11,  5, 12,  6,  6,  6, 12,  6,  4,
        5,  9,  6, 11,  6,  6, 11, 12, 12,  4, 12,  4,  4,  4,  5,  6,  4,
        4, 12,  4,  6,  4,  6, 11,  6,  4,  5,  4,  6,  3,  6, 12,  4,  4,
       12,  4,  4,  6,  4,  4,  3,  4,  4,  4,  4,  5,  4,  4,  9,  6,  4,
        4,  9, 13, 12,  4,  4, 12,  3, 12, 11,  4,  6, 11, 12,  4,  4,  3,
       12,  9,  4, 12,  2, 11, 12, 11,  6,  3,  4,  3, 12,  6, 12, 12,  6,
        4,  4,  4,  4,  6, 12,  9,  3,  6, 12,  6,  4,  6,  9,  4,  3,  4,
        4,  4, 11,  4,  4

### End ###